In [76]:
import pandas as pd
df = pd.read_csv(r'C:\\narod_without_html.csv', encoding='utf-8')

In [77]:
from string import punctuation

punctuation += '«»—…“”"|~'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~«»—…“”"|~'

очистка текста: удаление ссылок, адресов почт, добавление пробелов в "слипшиеся" слова

In [79]:
import re
def cleaned_text(text):
    cleaned_from_n = re.sub('[\n]+', ' ', text)
    cleaned_from_x = re.sub('\xa0', ' ', cleaned_from_n)
    cleaned_from_pocht = re.sub('[a-zA-Z]+@[a-zA-Z]+|&#769;|[XVI]+', '', cleaned_from_x)
    cleaned_from_web = re.sub('[a-zA-Z]+://[a-zA-Z\.\-\//?=\d&;]+[a-zA-Z\.\-\/?=\d&]*|://[a-zA-Z\.\-\//?=\d&;]+[a-zA-Z\.\-\/?=\d&]*|www[a-zA-Z.-/?=\d]+|<[/a-zA-Z\->"\.///:_/=\d\s]+/>|<[/a-zA-Z\->"\.///:_/=\d\s]+>|<[a-zA-Z]+|[a-zA-Z]+="[a-zA-Z-\d\/#&;=]+|[_a-zA-Z\d-]"|/[\d]+|[/>]|[\d]{2}:[\d]{2}:[\d]{2}|_[a-zA-Z]+_[a-zA-Z]+_[a-zA-Z]+_[a-zA-Z]+_[a-zA-Z]+|_[a-zA-Z]+_[a-zA-Z]+_[a-zA-Z]+_[a-zA-Z]+|-\d+-\d+-\d+', '', cleaned_from_pocht)
    added_spaces = re.sub('([а-яА-ЯёЁ])([!?:;»"\.\(\),])([а-яА-ЯёЁ])', r'\1\2 \3', cleaned_from_web)
    added_spaces_1 = re.sub('([а-яА-ЯёЁ])([«“])([а-яА-ЯёЁ])', r'\1 \2\3', added_spaces)
    added_spaces_2 = re.sub('([а-яА-ЯёЁ])([—])([а-яА-ЯёЁ])', r'\1 \2\3', added_spaces_1)
    cleaned_from_spaces = re.sub('[\s]{2,}', ' ', added_spaces_2).strip()

    return cleaned_from_spaces

заполнение нулей

In [80]:
df['page_text'] = df['page_text'].fillna("Не указано")

применение функции очистки

In [81]:
df['page_text'] = df['page_text'].apply(cleaned_text)

функция появилась в результате ручного исследования отрицательных кейсов

In [82]:
def dlya_mata(text):
    without_1 = re.sub('[а-яА-ЯёЁ]+[(]|:[а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+!|_[а-яА-ЯёЁ]+|[|][а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+[|]|[,][а-яА-ЯёЁ]+»|[*]При|[А-Яа-я][\'`][А-Яа-я]+|Назад<<<|Рассылки@|[а-яА-ЯёЁ]\\[а-яА-ЯёЁ]|[а-яА-ЯёЁ]\[[а-яА-ЯёЁ]|[а-яА-ЯёЁ]<[а-яА-ЯёЁ]|[а-яА-ЯёЁ]\)[а-яА-ЯёЁ]|[а-яА-ЯёЁ]\+[а-яА-ЯёЁ]|[а-яА-ЯёЁ]\][а-яА-ЯёЁ]|[а-яА-ЯёЁ]+@[а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+\$[а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+\&[а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+\^[а-яА-ЯёЁ]+|[а-яА-ЯёЁ]+\*[А-ЯЁ]', ' ', text)

    return without_1


In [83]:
df['page_text'] = df['page_text'].apply(dlya_mata)

In [86]:
def fix_text(text):
    separators = ['):%', '^;', '$$', '@«', 'дело%По', '»@', '^!|[', '^^', '?*', ':*', '*[*', '\\']
    
    for sep in separators:
        text = text.replace(sep, ' ') 
    
    return ' '.join(text.split()) 

df['page_text'] = df['page_text'].apply(fix_text)

In [87]:
from string import punctuation
punctuation += '«»…“”—'
results = []
for idx, i in enumerate(df['page_text']):
    if len(i) == 0:
        continue
    matches = re.sub('[ПНЖЕ][*]{1,3}', '', i)
    matchess = re.findall(
            r'[А-Яа-яёЁ]{1,3}[#$%&@^]{1,6}[А-Яа-яёЁ]+|[\/>\\\_|]+[А-Яа-яёЁ]+|[А-Яа-яёЁ]+[\/>\\|]+|\s[Жж][*]{1,4}[а-яА-ЯёЁ]{1,4}\s*|\s[о][х]{0,3}[*]{1,4}[а-яА-ЯёЁ]{0,6}\s|\s[Нн][иахс]{0,3}[*]{1,4}[а-яА-ЯёЁ]{0,4}\s|\s[гГ][*][а-яА-ЯёЁ]{1,4}|\s[Бб][л]{0,3}[*!]{1,4}[а-яА-ЯёЁ]{0,4}\s|\s[Пп][иозс]{0,4}[*]{1,4}[думастый]{0,4}\s*[!]*|\s[Ее][б]*[*]{1,4}[а-яА-ЯёЁ]{0,4}\s*|[на]*\s[Хх][*]{1,4}[а-яА-ЯёЁ]{0,4}\s|тр[*]{1,2}ть|обоср[*]{1,2}т*ься|подъё[*]ывать|З[*]{2}бало|славяноп*рдами', matches) 
        
    if not matchess:  
        continue
    results.append({'index': idx, 'matches': matchess})    
    print(matchess)

['чм###иками']
['П##дец']
[' пиз****м ']
['нах#й']
['БЛ#$Ь']
[' ж*па ', ' ж*пу']
[' г*вном', ' г*вно', ' Г*вно']
[' на*** ']
['с^%$ый', 'Ох@#нный']
[' е**ть ']
[' х*й ', ' х*й ', ' х* ', 'пи%добол', 'пи#де', 'п%$#р']
['о@#&%л']
['Оху%вший']
[' е**ть ', 'на х** ', ' пи**ду', ' бл***ми ', ' б**! ', ' б**! ', ' б**! ', ' е***ая', 'на х** ']
['Б%яяяяя']
[' п*']
['тр**ть']
['тр**ть', 'обоср*ться']
[' п*** ']
['на х* ']
['подъё*ывать']
[' еб*т']
['др#чат']
[' по**й']
[' нас***и ']
['подъё*ывать']
[' ох** ']
[' Бл**ь ']
[' Ни***я ']
['З**бало']
['за#бался']
['очк#а']
[' х*ю ']
[' на*** ']
['обоср**ься']
[' е*ать ']
['ж#пы']
[' пи***ас', ' пи*** !!!', ' пи*** !!!', ' пи***сы ']
[' пси***т ']


In [104]:
hhh = re.findall(r'[а-яА-ЯёЁ]+\%', df['page_text'][26064])
hh = re.findall(r'х\%\$', df['page_text'][15660])
ps = re.findall(r'на@#', df['page_text'][11724])
lis = [15635, 27465, 15660, 15674, 27451, 24743]
aa_ab = []
for i in lis:
    aa_ab += re.findall(r'[а-яА-ЯёЁ]+@[а-яА-ЯёЁ]+', df['page_text'][i])
resh = re.findall(r'#[а-яА-ЯёЁ]+', df['page_text'][19033])
dd = re.findall(r'.{3}[а-яА-ЯёЁ]+\$.{5}', df['page_text'][15635])
bb = re.findall(r'х\^\%|х\%\$', df['page_text'][15660])
ff = re.findall(r'[а-яА-ЯёЁ]+\$.{2}', df['page_text'][15660])

In [203]:
aa_ab = ['#опа', 'ох@ел', 'на@й', 'х@м', 'х@ем', 'х@ем', 'о@уел', 'ж@пу', 'МуД@К']

['#опа', 'ох@ел', 'на@й', 'х@м', 'х@ем', 'х@ем', 'о@уел', 'ж@пу', 'МуД@К']

In [216]:
chu = re.findall(r'[а-яА-ЯёЁ]+\$.{1}', df['page_text'][32961])
matchesss = re.findall('\s[*]+[А-Яа-яёЁ]+', df['page_text'][36496])
matchesss += re.findall('\s[*]+[А-Яа-яёЁ]+', df['page_text'][25255])
matchesss
a = chu + aa_ab + ff + ps + dd + hh + hhh + matchesss + bb
a

['х$&',
 '#опа',
 'ох@ел',
 'на@й',
 'х@м',
 'х@ем',
 'х@ем',
 'о@уел',
 'ж@пу',
 'МуД@К',
 'на@#',
 'Ни ху$ себе',
 'х%$',
 'бл%',
 ' **ля',
 ' **пу',
 ' ****ься',
 'х^%',
 'х%$']

In [94]:
from collections import defaultdict

In [97]:
censored_words = {
    'чм###иками': 'чмошники',
    'П##дец': 'пиздец',
    'пиз****м': 'пиздец',
    'нах#й': 'нахуй',
    'БЛ#$Ь': 'бля(ть)',
    'Б***': 'бля(ть)',
    'ж*па': 'жопа',
    'ж*пу': 'жопа',
    'г*вном': 'говно',
    'г*вно': 'говно',
    'Г*вно': 'говно',
    'на***': 'нахуй',
    'с^%$ый': 'срать',
    'Ох@#нный': 'охуеть',
    'е**ть': 'ебать',
    'х*й': 'хуй',
    'х*': 'хуй',
    'пи%добол': 'пиздабол',
    'пи#де': 'пизда',
    'п%$#р': 'пидор',
    'о@#&%л': 'охуеть',
    'Оху%вший': 'охуеть',
    'пи**ду': 'пизда',
    'бл***ми': 'блядь',
    'б**!': 'бля(ть)',
    'е***ая': 'ебать',
    'Б%яяяяя': 'бля(ть)',
    'п*': 'пизда',
    'тр**ть': 'трахать',
    'обоср*ться': 'срать',
    'п***': 'пизда',
    'на х*': 'нахуй',
    'подъё*ывать': 'подъёбывать',
    'еб*т': 'ебать',
    'др#чат': 'дрочить',
    'по**й': 'похуй',
    'нас***и': 'срать',
    'ох**': 'охуеть',
    'Бл**ь': 'бля(ть)',
    'Ни***я': 'нихуя',
    'З**бало': 'заебать',
    'за#бался': 'заебать',
    'очк#а': 'очко',
    'х*ю': 'хуй',
    'обоср**ься': 'срать',
    'о****енение': 'охуеть',
    'е*ать': 'ебать',
    'ж#пы': 'жопа',
    'пи***ас': 'пидор',
    'пи*** !!!': 'пидор',
    'пи***сы': 'пидор',
    'пси***т': 'психопат',
    'х$&': 'хуй',
    '#опа': 'жопа',
    'ох@ел': 'охуеть',
    'на@й': 'нахуй',
    'х@м': 'хуй',
    'х@ем': 'хуй',
    'о@уел': 'охуеть',
    'ж@пу': 'жопа',
    'МуД@К': 'мудак',
    'на@#': 'нахуй',
    'Ни ху$ себе': 'хуй',
    'х%$': 'хуй',
    'бл%': 'бля(ть)',
    '**ля': 'бля(ть)',
    '**пу': 'жопа',
    '****ься': 'ебать',
    'х^%': 'хуй',
    'х%$': 'хуй'
}
with open('ostorozhno_mat.txt', 'w') as f:
    f.write(str(censored_words))


In [106]:
from collections import defaultdict

In [109]:
slovar_mata = defaultdict(list)

for cenz, mat in censored_words.items():
    normalized_mat = mat.lower()  
    slovar_mata[normalized_mat].append(cenz)

sorted_mat = sorted(slovar_mata.items(), key=lambda x: len(x[1]))

table = []
for nf, variants in sorted_mat:
    table.append({
        'Начальная форма': nf,
        'Варианты цензуры': ', '.join(variants)
    })

df = pd.DataFrame(table)

In [65]:
df

,Начальная форма,Варианты цензуры
0,хуй,"х*й, х*, х*ю, х$&, х@м, х@ем, Ни ху$ себе, х%$, х^%"
1,бля(ть),"БЛ#$Ь, Б***, б**!, Б%яяяяя, Бл**ь, бл%, **ля"
2,охуеть,"Ох@#нный, о@#&%л, Оху%вший, ох**, о****енение, ох@ел, о@уел"
3,жопа,"ж*па, ж*пу, ж#пы, #опа, ж@пу, **пу"
4,нахуй,"нах#й, на***, на х*, на@й, на@#"
5,ебать,"е**ть, е***ая, еб*т, е*ать, ****ься"
6,срать,"с^%$ый, обоср*ться, нас***и, обоср**ься"
7,пизда,"пи#де, пи**ду, п*, п***"
8,пидор,"п%$#р, пи***ас, пи*** !!!, пи***сы"
9,говно,"г*вном, г*вно, Г*вно"
